# Tutorial 6: Function Secret Sharing
In our library, we also support Function Secret Sharing, which refers to Boyle et al. 's paper: Function Secret Sharing: Improvements and Extensions.2016; Function Secret Sharing for Mixed-Mode and Fixed-Point Secure Computation.2021
Function secret sharing is mainly the secret sharing of three functions, distributed point function (DPF), distributed comparison function (DCF) and distributed interval containment function (DICF).
In addition, we also refer to paper: ``Grotto: Screaming fast (2+ 1)-PC for ℤ2n via (2, 2)-DPFs`` and paper: ``SIGMA: Secure GPT Inference with Function Secret Sharing`` to implement the functionality of DICF by DPF

## DPF
Definition：
$$
f_{\alpha,\beta}(x)=\begin{cases}
\beta, & \text {if x=$\alpha$} \\
0, & \text {else}
\end{cases}
$$
We need to ask for the value of the function at the x position, and at the same time we need to hide $\alpha$ and $\beta$. The scheme adopted is to have the DPF key over $\alpha$ and $\beta$ generated by a trusted third party, and then to distribute the key to the two parties, who each use the key to calculate the value of the function when the independent variable is x.

Regarding the operation of shape transformation on x, we explain as follows: Since the supported representation data only supports 32 or 64 bits, to represent 128-bit data, you need the data of 4 int32s or 2 int64s to be concatenated together. This is the case for the random number ``s`` required by DPFKey. So we need to convert x to a column vector so that we don't have dimension mismatches when we perform subsequent multiplications.

In [1]:
# import the libraries
from NssMPC.primitives import DPF, DCF, DICF, GrottoDICF, SigmaDICF
from NssMPC.primitives.secret_sharing.function import DPFKey, DICFKey, GrottoDICFKey, SigmaDICFKey
from NssMPC import RingTensor

num_of_keys = 10  # We need a few keys for a few function values, but of course we can generate many keys in advance.

# generate keys in offline phase
# set alpha and beta
alpha = RingTensor.convert_to_ring(5)
beta = RingTensor.convert_to_ring(1)

Key0, Key1 = DPFKey.gen(num_of_keys=num_of_keys, alpha=alpha, beta=beta)
# online phase
# generate some values what we need to evaluate
x = RingTensor.convert_to_ring([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
shape=x.shape
x = x.reshape(-1,1)

# Party 0:
res_0 = DPF.eval(x=x, keys=Key0, party_id=0)

# Party 1:
res_1 = DPF.eval(x=x, keys=Key1, party_id=1)

# restore result
res = res_0 + res_1
res=res.reshape(shape)
print(res)

# DPF supports the calculation of values on different sized rings.
# To implement related operations, we need to manually change the ring in which the value is located, i.e. the bit_len of the value.
# For example, all calculations are performed in a ring of size 2^8
alpha = alpha.convert_to_range(bit_len=8)  # can use `alpha.convert_to_range(bit_len=8)` directly
x = x.convert_to_range(bit_len=8)

# generate keys in offline phase
Key0, Key1 = DPFKey.gen(num_of_keys=num_of_keys, alpha=alpha, beta=beta)

# online phase
# Party 0:
res_0 = DPF.eval(x=x, keys=Key0, party_id=0)

# Party 1:
res_1 = DPF.eval(x=x, keys=Key1, party_id=1)

# restore result
res = res_0 + res_1
res=res.reshape(shape)
print(res)

RingTensor
 value:tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0], device='cuda:0') 
 dtype:int 
 scale:1
RingTensor
 value:tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0], device='cuda:0') 
 dtype:int 
 scale:1


## DCF
Definition：
$$
f_{\alpha,\beta}(x)=\begin{cases}
\beta, & \text {if x < $\alpha$} \\
0, & \text {else}
\end{cases}
$$
The method for computing the DCF value is similar to that of DPF.

In [2]:
# import the libraries
from NssMPC import RingTensor

num_of_keys = 10  # We need a few keys for a few function values, but of course we can generate many keys in advance.

# generate keys in offline phase
# set alpha and beta
alpha = RingTensor.convert_to_ring(5)
beta = RingTensor.convert_to_ring(1)

Key0, Key1 = DCF.gen(num_of_keys=num_of_keys, alpha=alpha, beta=beta)

# online phase
# generate some values what we need to evaluate
x = RingTensor.convert_to_ring([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# Party 0:
res_0 = DCF.eval(x=x, keys=Key0, party_id=0)

# Party 1:
res_1 = DCF.eval(x=x, keys=Key1, party_id=1)

# restore result
res = res_0 + res_1
print(res)

RingTensor
 value:tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0], device='cuda:0') 
 dtype:int 
 scale:1


## DICF
definition：
$$
f_{p,q}(x)=\begin{cases}
1, & \text {if p$\leq$x $\leq$ q} \\
0, & \text {else}
\end{cases}
$$
We set the function value to 1 when x falls within the range of p and q. With DICF, we can achieve a comparison between two numbers. For two numbers, x and y, if the difference between them falls within the positive range, it indicates that x is greater than y.
Now we will demonstrate how to compute the value of the DICF for a given input x. However, unlike DPF and DCF, the DICF hides the information of the input x, rather than the information of the upper and lower bounds. This is done to serve the purpose of performing comparison.

The library supports three DICF operations: 2021 Function secret sharing, GROTTO, SIGMA.

In [3]:
# import the libraries
from NssMPC import RingTensor

# generate key in offline phase
num_of_keys = 10
down_bound = RingTensor(3)
upper_bound = RingTensor(7)

Key0, Key1 = DICFKey.gen(num_of_keys=num_of_keys, down_bound=down_bound, upper_bound=upper_bound)

# evaluate x in online phase
# generate some values what we need to evaluate
x = RingTensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
x_shift = x + Key0.r_in.reshape(x.shape) + Key1.r_in.reshape(x.shape)

# online phase
# Party 0:
res_0 = DICF.eval(x_shift=x_shift, keys=Key0, party_id=0, down_bound=down_bound, upper_bound=upper_bound)

# Party 1:
res_1 = DICF.eval(x_shift=x_shift, keys=Key1, party_id=1, down_bound=down_bound, upper_bound=upper_bound)

# restore result
res = res_0 + res_1
print(res)

RingTensor
 value:tensor([0, 0, 1, 1, 1, 1, 1, 0, 0, 0], device='cuda:0') 
 dtype:int 
 scale:1


#### GROTTO
The paper GROTTO proposes a scheme to implement DICF faster with prefix sum and DPF. The following is the sample code. It should be noted that although the result evaluated by this scheme is 0,1, it actually corresponds to bool values False and True, that is to say, the result of this scheme is a Boolean secret share value, while the previous schemes are arithmetic secret share values.

In [4]:
# import the libraries
from NssMPC import RingTensor

# generate key in offline phase
num_of_keys = 10
down_bound = RingTensor(3)
upper_bound = RingTensor(7)
beta = RingTensor(1)

Key0, Key1 = GrottoDICFKey.gen(num_of_keys=num_of_keys, beta=beta)

# evaluate x in online phase
# generate some values what we need to evaluate
x = RingTensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
x_shift = Key0.r_in.reshape(x.shape) + Key1.r_in.reshape(x.shape) - x

# online phase
# Party 0:
res_0 = GrottoDICF.eval(x_shift=x_shift, key=Key0, party_id=0, down_bound=down_bound, upper_bound=upper_bound)

# Party 1:
res_1 = GrottoDICF.eval(x_shift=x_shift, key=Key1, party_id=1, down_bound=down_bound, upper_bound=upper_bound)

# restore result
res = res_0 ^ res_1
print(res)

RingTensor
 value:tensor([0, 0, 1, 1, 1, 1, 0, 0, 0, 0], device='cuda:0') 
 dtype:int 
 scale:1


#### SIGMA
The paper SIGMA proposes a DReLU scheme based on DPF. Here we show how to use such method to implement DReLU function. Note that the result of this scheme is also a Boolean secret share value.

$$
\text{DReLU}(x)=\begin{cases}
1, & \text {if x>=0} \\
0, & \text {else}
\end{cases}
$$


In [5]:
# import the libraries
from NssMPC import RingTensor

# generate key in offline phase
num_of_keys = 10

Key0, Key1 = SigmaDICFKey.gen(num_of_keys=num_of_keys)

# evaluate x in online phase
# generate some values what we need to evaluate
x = RingTensor([-5, -4, -3, -2, -1, 0, 1, 2, 3, 4])
x_shift = Key0.r_in.reshape(x.shape) + Key1.r_in.reshape(x.shape) + x

# online phase
# Party 0:
res_0 = SigmaDICF.eval(x_shift=x_shift, key=Key0, party_id=0)

# Party 1:
res_1 = SigmaDICF.eval(x_shift=x_shift, key=Key1, party_id=1)

# restore result
res = res_0 ^ res_1
print(res)

# 和DPF一样，SIGMADICF支持不同大小的环上的计算。
# The same as DPF, SIGMADICF supports the calculation of values on different rings.
# generate keys in offline phase
Key0, Key1 = SigmaDICFKey.gen(num_of_keys=num_of_keys, bit_len=8)  # set the bit_len to 8

# evaluate x in online phase
x_shift = Key0.r_in.reshape(x.shape) + Key1.r_in.reshape(x.shape) + x
x_shift = x_shift.convert_to_range(bit_len=8)  # need to convert input to the corresponding ring

# Party 0:
res_0 = SigmaDICF.eval(x_shift=x_shift, key=Key0, party_id=0)

# Party 1:
res_1 = SigmaDICF.eval(x_shift=x_shift, key=Key1, party_id=1)

# restore result
res = res_0 ^ res_1
print(res)

RingTensor
 value:tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1], device='cuda:0') 
 dtype:int 
 scale:1
RingTensor
 value:tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1], device='cuda:0') 
 dtype:int 
 scale:1


## The application of FSS
As mentioned before, DICF can be used for number comparison. If you want to use DICF for number comparison, please change `GE_TYPE` in`./config/configs.py` to `DICF`, and then refer to the method in Tutorial_2 to achieve it. We also talked about an improved FSS method in Tutorial_0. If you want to use this method for number comparison, change `GE_TYPE` to `PPQ` or `SIGMA` and then refer to Tutorial_2.